In [1]:
import json

# Đường dẫn đến các file dữ liệu
train_file_path = 'Project2_Data/train.json'
test_file_path = 'Project2_Data/test.json'

# Hàm để đọc dữ liệu từ file JSON
def read_json(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        data = json.load(file)
    return data

# Đọc dữ liệu từ file
train_data = read_json(train_file_path)
test_data = read_json(test_file_path)

# Hiển thị cấu trúc của dữ liệu
train_data_structure = {
    "Total Examples in Train": len(train_data),
    "First Example": train_data[0],
    "Context Length": len(train_data[0]['context']),
    "Total Questions in First Example": len(train_data[0]['qas']),
    "First Question": train_data[0]['qas'][0]
}

test_data_structure = {
    "Total Examples in Test": len(test_data),
    "First Example": test_data[0],
    "Context Length": len(test_data[0]['context']),
    "Total Questions in First Example": len(test_data[0]['qas']),
    "First Question": test_data[0]['qas'][0]
}

train_data_structure


{'Total Examples in Train': 12000,
 'First Example': {'context': 'Về mặt kiến \u200b\u200btrúc, trường có một nhân vật Công giáo. Trên đỉnh mái vòm vàng tòa nhà của chính là bức tượng vàng của Đức Trinh Nữ Maria. Ngay ở phía trước của Tòa nhà Chính và phải đối mặt với nó, là một bức tượng đồng của Chúa Kitô với vũ khí upraised với truyền thuyết "Venite rao nhớ Omnes". Bên cạnh Tòa nhà Chính là Vương Cung Thánh Đường Thánh Tâm Chúa. Ngay phía sau nhà thờ là Grotto, một nơi Marian cầu nguyện và suy. Nó là một bản sao của hang động ở Lourdes, Pháp nơi Đức Trinh Nữ Maria hiện ra với reputedly Saint Bernadette Soubirous trong năm 1858. Vào cuối của ổ đĩa chính (và trong một đường thẳng nối thông qua 3 tượng và Dome Gold), là một bức tượng đá hiện đại đơn giản của Mary. .',
  'qas': [{'id': '5733be284776f4190066117e',
    'question': 'Có gì ngồi trên đầu trang của Tòa nhà Chính tại Notre Dame?',
    'answers': [{'text': 'bức tượng vàng của Đức Trinh Nữ Maria',
      'answer_start': 98}],
   

In [2]:
from transformers import XLNetTokenizerFast
import torch
from torch.utils.data import DataLoader, TensorDataset

# Sử dụng XLNetTokenizerFast
tokenizer = XLNetTokenizerFast.from_pretrained('xlnet-base-cased')

def encode_data(data):
    input_ids = []
    attention_masks = []
    start_positions = []
    end_positions = []
    is_impossible = []

    for item in data:
        context = item['context']
        for qa in item['qas']:
            question = qa['question']
            # Encode câu hỏi và context
            encoded = tokenizer.encode_plus(question, context, max_length=512, truncation=True, padding='max_length', return_tensors='pt', return_offsets_mapping=True)
            input_id = encoded['input_ids']
            attention_mask = encoded['attention_mask']
            offset_mapping = encoded['offset_mapping'][0].tolist()

            # Mặc định giá trị cho start và end position
            start_pos = 0
            end_pos = 0

            # Xử lý câu trả lời
            if not qa['is_impossible']:
                answer = qa['answers'][0]['text']
                start_char = qa['answers'][0]['answer_start']
                end_char = start_char + len(answer)

                # Tìm vị trí token của câu trả lời
                sequence_ids = encoded.sequence_ids()
                for i, (offset_start, offset_end) in enumerate(offset_mapping):
                    if sequence_ids[i] == 1:  # Phần của context
                        if start_char >= offset_start and end_char <= offset_end:
                            start_pos = i
                            end_pos = i
                            break
                        elif start_char < offset_end and end_pos == 0:
                            start_pos = i
                        if end_char > offset_start and start_pos != 0:
                            end_pos = i

            input_ids.append(input_id)
            attention_masks.append(attention_mask)
            start_positions.append(start_pos)
            end_positions.append(end_pos)
            is_impossible.append(int(qa['is_impossible']))
    
    # Chuyển đổi lists thành tensors
    input_ids = torch.cat(input_ids, dim=0)
    attention_masks = torch.cat(attention_masks, dim=0)
    start_positions = torch.tensor(start_positions, dtype=torch.long)
    end_positions = torch.tensor(end_positions, dtype=torch.long)
    is_impossible = torch.tensor(is_impossible, dtype=torch.long)  # Thay đổi dtype sang torch.long
    
    return input_ids, attention_masks, start_positions, end_positions, is_impossible

# Tiền xử lý dữ liệu
# train_data và test_data đã được định nghĩa từ bước trước
train_input_ids, train_attention_masks, train_start_positions, train_end_positions, train_is_impossible = encode_data(train_data)
test_input_ids, test_attention_masks, test_start_positions, test_end_positions, test_is_impossible = encode_data(test_data)

# Tạo TensorDataset và DataLoader
batch_size = 6
train_dataset = TensorDataset(train_input_ids, train_attention_masks, train_start_positions, train_end_positions, train_is_impossible)
test_dataset = TensorDataset(test_input_ids, test_attention_masks, test_start_positions, test_end_positions, test_is_impossible)

train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=batch_size)
test_dataloader = DataLoader(test_dataset, shuffle=False, batch_size=batch_size)


c:\Users\Huy Anh\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# In ra một số mẫu dữ liệu từ train dataset để kiểm tra
def print_sample_data(input_ids, attention_masks, start_positions, end_positions, is_impossible, num_samples=3):
    for i in range(num_samples):
        print(f"Sample {i+1}:")
        print("Input IDs:", input_ids[i])
        print("Attention Masks:", attention_masks[i])
        print("Start Position:", start_positions[i])
        print("End Position:", end_positions[i])
        print("Is Impossible:", is_impossible[i])
        print("\n")

# Chọn một số mẫu ngẫu nhiên để kiểm tra
import random

random_indices = random.sample(range(len(train_input_ids)), 3)
sample_input_ids = train_input_ids[random_indices]
sample_attention_masks = train_attention_masks[random_indices]
sample_start_positions = train_start_positions[random_indices]
sample_end_positions = train_end_positions[random_indices]
sample_is_impossible = train_is_impossible[random_indices]

print_sample_data(sample_input_ids, sample_attention_masks, sample_start_positions, sample_end_positions, sample_is_impossible)


Sample 1:
Input IDs: tensor([  574,  2582,  2483,    17,   267,  2762,  2721,  5835, 10479,   150,
           17,     0,    93,    17,  2970, 23444,  9712,    17,  1598,   430,
        17314, 13781,   369,    82,     4,    17,     0,   155,  2483,    17,
         7575,    17,  6684,   150, 10479,   150,    17,  3192,    17, 13836,
           98,   102,   627,    17,  3525,   101,    17,     0,    93,    17,
         2970, 23444,  9712,    17,  1598,   430, 17314, 13781,   369,    17,
         2853, 19865,    13, 26052,    17,  1598,    17,    23,  2168,    17,
            0,   155, 10479,   150,    17,  3192,    17, 13836,    98,   102,
          627,    17,  3525,   101,   128,  7735, 18551,    23,    17,    46,
        16983,   425,   963,     9,   330,  2996,  2003,  2874,    66,  4931,
           19,   541,   815,    13, 26214,    19,    17,  4063,  5835,  6938,
          150,    17, 13836,    98,   102,   627,  3074,  2505,    17,  3525,
          101,    17,   267,  2762,  2721, 

In [4]:
# Chuẩn Bị Mô Hình
# Khởi tạo tokenizer và mô hình

from transformers import XLNetForQuestionAnsweringSimple, AdamW, XLNetTokenizerFast

tokenizer = XLNetTokenizerFast.from_pretrained('xlnet-base-cased')
model = XLNetForQuestionAnsweringSimple.from_pretrained('xlnet-base-cased')

Some weights of XLNetForQuestionAnsweringSimple were not initialized from the model checkpoint at xlnet-base-cased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
# Cấu Hình Optimizer
optimizer = AdamW(model.parameters(), lr=5e-5)

c:\Users\Huy Anh\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\optimization.py:457: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [6]:
# Chuẩn Bị Chu kỳ Huấn Luyện
from transformers import get_linear_schedule_with_warmup

epochs = 2
total_steps = len(train_dataloader) * epochs

scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)

In [7]:
from torch.utils.data import  random_split

train_size = int(0.8 * len(train_dataset))  # 80% dữ liệu cho huấn luyện
validation_size = len(train_dataset) - train_size  # 20% còn lại cho validation

# Chia dataset
train_dataset, validation_dataset = random_split(train_dataset, [train_size, validation_size])

# Tạo DataLoader cho phần train và validation
train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=8)
validation_dataloader = DataLoader(validation_dataset, shuffle=False, batch_size=8)


In [8]:
import time
import datetime

# Thiết lập thiết bị
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Hàm để định dạng thời gian trôi qua
def format_time(elapsed):
    return str(datetime.timedelta(seconds=int(round((elapsed)))))

# Khởi tạo biến để lưu trọng số tốt nhất
best_loss = float('inf')
best_params_path = 'best_params.txt'  # Đường dẫn tệp để lưu thông số tốt nhất

# Bắt đầu huấn luyện
for epoch_i in range(epochs):
    print(f"\n======== Epoch {epoch_i + 1} / {epochs} ========")
    print("Training...")

    t0 = time.time()
    total_train_loss = 0
    model.train()

    for step, batch in enumerate(train_dataloader):
        if step % 200 == 0 and not step == 0:
            elapsed = format_time(time.time() - t0)
            print(f"  Batch {step} of {len(train_dataloader)}. Elapsed: {elapsed}.")

        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_attention_mask, b_start_positions, b_end_positions = batch[:4]

        model.zero_grad()
        outputs = model(b_input_ids, attention_mask=b_attention_mask, start_positions=b_start_positions, end_positions=b_end_positions)
        loss = outputs.loss
        total_train_loss += loss.item()

        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()

        # Cập nhật và lưu thông số nếu tốt hơn
        if loss.item() < best_loss:
            best_loss = loss.item()
            with open(best_params_path, 'w') as f:
                for name, param in model.named_parameters():
                    f.write(f"{name}: {param.data}\n")
            print(f"Updated best model parameters at batch {step} with loss {best_loss}")

    avg_train_loss = total_train_loss / len(train_dataloader)
    training_time = format_time(time.time() - t0)
    print(f"\n  Average training loss: {avg_train_loss:.2f}")
    print(f"  Training epoch took: {training_time}")

print("\nTraining complete!")


======== Epoch 1 / 2 ========
Training...
Updated best model parameters at batch 0 with loss 6.687341690063477
Updated best model parameters at batch 1 with loss 6.386669158935547
Updated best model parameters at batch 2 with loss 5.938324928283691
Updated best model parameters at batch 5 with loss 5.717538833618164
Updated best model parameters at batch 6 with loss 5.667553901672363
Updated best model parameters at batch 10 with loss 5.592215538024902
Updated best model parameters at batch 12 with loss 5.477365493774414
Updated best model parameters at batch 13 with loss 5.025311470031738
Updated best model parameters at batch 24 with loss 4.970441818237305
Updated best model parameters at batch 25 with loss 4.935801029205322
Updated best model parameters at batch 30 with loss 4.126447677612305
Updated best model parameters at batch 56 with loss 4.048372745513916
Updated best model parameters at batch 75 with loss 3.972717761993408
Updated best model parameters at batch 94 with loss 

In [9]:
import numpy as np

def format_time(elapsed):
    return str(datetime.timedelta(seconds=int(round((elapsed)))))

# Chuyển mô hình sang chế độ đánh giá
model.eval()

# Theo dõi các biến để tính toán độ chính xác và loss
total_eval_accuracy = 0
total_eval_loss = 0
nb_eval_steps = 0

# Ghi lại thời gian bắt đầu
t0 = time.time()

# Không cần tính toán hoặc lưu gradient
with torch.no_grad():
    for batch in test_dataloader:
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_attention_mask, b_start_positions, b_end_positions = batch[:4]

        outputs = model(b_input_ids, attention_mask=b_attention_mask, start_positions=b_start_positions, end_positions=b_end_positions)
        
        loss = outputs.loss
        total_eval_loss += loss.item()

        # Lấy logit dự đoán cho vị trí bắt đầu và kết thúc
        start_logits = outputs.start_logits.detach().cpu().numpy()
        end_logits = outputs.end_logits.detach().cpu().numpy()
        start_positions = b_start_positions.to('cpu').numpy()
        end_positions = b_end_positions.to('cpu').numpy()

        # Chuyển logit thành index dự đoán
        start_preds = np.argmax(start_logits, axis=-1)
        end_preds = np.argmax(end_logits, axis=-1)

        # Tính độ chính xác cho batch
        eval_accuracy = (np.sum(start_preds == start_positions) + np.sum(end_preds == end_positions)) / (len(start_preds) + len(end_preds))
        total_eval_accuracy += eval_accuracy

# Tính và in ra các thông số đánh giá
avg_loss = total_eval_loss / len(test_dataloader)
avg_accuracy = total_eval_accuracy / len(test_dataloader)
total_time = format_time(time.time() - t0)  

print(f"Test Accuracy: {avg_accuracy:.2f}")
print(f"Average Test Loss: {avg_loss:.2f}")
print(f"Testing took: {total_time}")


Test Accuracy: 0.61
Average Test Loss: 1.54
Testing took: 2:02:06
